<a href="https://colab.research.google.com/github/MiraDetrasTuyo/Coursera_Capstone/blob/main/Coursera-Capstone-Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Question

In [1]:
# Importing libraries

import pandas as pd

In [2]:
# Scrapping the table from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

column_name = ['Postal Code', 'Borough', 'Neighbourhood']

for scraped_df in dfs:
    if list(scraped_df.columns.values) == column_name:
        df = scraped_df
        break

df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# Renaming the Headers
df.rename(columns={"Postal Code": "PostalCode"}, inplace = True)

# Dropping rows w/out a Bourough
df = df[df.Borough != "Not assigned"]

# If there is no Neighbourhood assigned: it will be the same as the Borough
for index, row in df.iterrows():
    if row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row['Borough']

df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Second Question

In [4]:
# We generate the coordinate dataframe

df_coord = pd.read_csv("http://cocl.us/Geospatial_data")
df_coord.rename(columns={"Postal Code": "PostalCode"}, inplace = True)
df_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# We merge both dataframes

df = df.merge(df_coord, how = 'left', on = 'PostalCode')
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Third Question

In [7]:
# We import the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

print('Libraries imported.')

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.
/bin/bash: conda: command not found
Libraries imported.
/bin/bash: conda: command not found
Libraries imported.


In [8]:
# Let's see how many unique values do we have
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [15]:
# We introduce the Foursquare API information

CODE = 'YGFEHA4LIYPOPAVCJGOP3VZJXD5EP01NWR2M5DL1CG4YAE0H#_=_'
CLIENT_ID = 'DV4Z5RO1FG2GFVSZE42KHY1QGZSQB5QB13S11OIGJS3L2KNI' # your Foursquare ID
CLIENT_SECRET = 'W3L3UVMRI5TUWSDZNGGMRY5DPYMJ0XK2NCR4NIOI4TCWN0Q0' # your Foursquare Secret
ACCESS_TOKEN = '5TQJIVKCA15AYK1MNAFT4GIJHBALP24XJEZ0GLCPIVZAZMGO' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DV4Z5RO1FG2GFVSZE42KHY1QGZSQB5QB13S11OIGJS3L2KNI
CLIENT_SECRET:W3L3UVMRI5TUWSDZNGGMRY5DPYMJ0XK2NCR4NIOI4TCWN0Q0
